In [ ]:
def as_list(x):
    if type(x) is list:
        return x
    else:
        return [x]

In [ ]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import numpy as np

# global variables
api_key = 'RGAPI-3f8d6099-ad69-4142-86ed-a2f6a71ad9d0'
watcher = LolWatcher(api_key)
my_region = 'euw1'


In [ ]:
# fetch matchlist by puuid
me = watcher.summoner.by_name(my_region, 'Kelody')
# me = watcher.summoner.by_name(my_region, 'daimox')
my_matches = watcher.match.matchlist_by_puuid("EUROPE", me['puuid'])
matches = []
matchTimeLine = []
participants = []


In [ ]:
# fetch game, participant and match timeline and transform into dataframe
for matchId in my_matches:
    match_detail = watcher.match.by_id("EUROPE", matchId)
    timeline = watcher.match.timeline_by_match('EUROPE', matchId)

    match = {}
    match['matchId'] = matchId
    match['gameDuration'] = match_detail['info']['gameDuration']
    match['gameType'] = match_detail['info']['gameType']
    match['gameVersion'] = match_detail['info']['gameVersion']
    matches.append(match)

    for index, row in enumerate(match_detail['info']['participants']):
        participants_row = {}
        participants_row['matchId'] = matchId
        participants_row['participant'] = index + 1
        participants_row['summonerName'] = row['summonerName']
        participants_row['teamPosition'] = row['teamPosition']
        participants_row['championName'] = row['championName']
        participants_row['kda'] = f"{row['kills']}/{row['deaths']}/{row['assists']}"
        participants_row['champLevel'] = row['champLevel']
        participants_row['totalDamageToChampions'] = row['totalDamageDealtToChampions']
        participants_row['totalDamageTaken'] = row['totalDamageTaken']
        participants_row['totalHeal'] = row['totalHeal']
        participants_row['totalMinionsKilled'] = row['totalMinionsKilled']
        participants_row['goldEarned'] = row['goldEarned']
        participants_row['summoner1'] = row['summoner1Id']
        participants_row['summoner2'] = row['summoner2Id']
        participants_row['item0'] = row['item0']
        participants_row['item1'] = row['item1']
        participants_row['item2'] = row['item2']
        participants_row['item3'] = row['item3']
        participants_row['item4'] = row['item4']
        participants_row['item5'] = row['item5']
        participants_row['item6'] = row['item6']
        participants_row['win'] = row['win']
        participants.append(participants_row)

    for row in timeline['info']['frames']:
        # frameCounter = 1
        # frame = row["{:02d}".format(frameCounter)]
        # frameCounter += 1
        for frameRow in row['events']:
            timeline_row = {}
            timeline_row['matchId'] = matchId
            eventCounter = 0
            # eventFrame = frameRow["{:02d}".format(eventCounter)]
            eventFrame = frameRow
            eventCounter += 1

            if eventFrame['type'] == 'CHAMPION_KILL':
                timeline_row['type'] = eventFrame['type']
                timeline_row['killerId'] = eventFrame['killerId']
                timeline_row['victimId'] = eventFrame['victimId']
                if 'assistingParticipantIds' in eventFrame:
                    timeline_row['assistingParticipantIds'] = eventFrame['assistingParticipantIds']
                if 'bounty' in eventFrame:
                    timeline_row['bounty'] = eventFrame['bounty']
                timeline_row['positionX'] = eventFrame['position']['x']
                timeline_row['positionY'] = eventFrame['position']['y']
                timeline_row['timestamp'] = eventFrame['timestamp']
                matchTimeLine.append(timeline_row)
            elif eventFrame['type'] == 'BUILDING_KILL':
                timeline_row['type'] = eventFrame['type']
                timeline_row['killerId'] = eventFrame['killerId']
                timeline_row['teamId'] = eventFrame['teamId']
                timeline_row['buildingType'] = eventFrame['buildingType']
                timeline_row['laneType'] = eventFrame['laneType']
                timeline_row['positionX'] = eventFrame['position']['x']
                timeline_row['positionY'] = eventFrame['position']['y']
                timeline_row['timestamp'] = eventFrame['timestamp']
                matchTimeLine.append(timeline_row)
            elif eventFrame['type'] == 'ITEM_PURCHASED':
                timeline_row['type'] = eventFrame['type']
                timeline_row['participantId'] = eventFrame['participantId']
                timeline_row['itemId'] = eventFrame['itemId']
                timeline_row['timestamp'] = eventFrame['timestamp']
                matchTimeLine.append(timeline_row)
            elif eventFrame['type'] == 'WARD_PLACED':
                timeline_row['type'] = eventFrame['type']
                timeline_row['wardType'] = eventFrame['wardType']
                timeline_row['creatorId'] = eventFrame['creatorId']
                timeline_row['timestamp'] = eventFrame['timestamp']
                matchTimeLine.append(timeline_row)
            elif eventFrame['type'] == 'WARD_KILL':
                timeline_row['type'] = eventFrame['type']
                timeline_row['wardType'] = eventFrame['wardType']
                timeline_row['killerId'] = eventFrame['killerId']
                timeline_row['timestamp'] = eventFrame['timestamp']
                matchTimeLine.append(timeline_row)
            elif eventFrame['type'] == 'LEVEL_UP':
                timeline_row['type'] = eventFrame['type']
                timeline_row['participantId'] = eventFrame['participantId']
                timeline_row['level'] = eventFrame['level']
                timeline_row['timestamp'] = eventFrame['timestamp']
                matchTimeLine.append(timeline_row)
            elif eventFrame['type'] == 'ELITE_MONSTER_KILL':
                timeline_row['type'] = eventFrame['type']
                timeline_row['killerId'] = eventFrame['killerId']
                timeline_row['teamId'] = eventFrame['killerTeamId']
                if 'assistingParticipantIds' in eventFrame:
                    timeline_row['assistingParticipantIds'] = eventFrame['assistingParticipantIds']
                timeline_row['positionX'] = eventFrame['position']['x']
                timeline_row['positionY'] = eventFrame['position']['y']
                timeline_row['timestamp'] = eventFrame['timestamp']
                timeline_row['monsterType'] = eventFrame['monsterType']
                matchTimeLine.append(timeline_row)


participantsDf = pd.DataFrame(participants)
matchesDf = pd.DataFrame(matches)
matchTimelineDf = pd.DataFrame(matchTimeLine)
print(participantsDf)
print(matchesDf)
print(matchTimelineDf)


In [ ]:
kelody = participantsDf[participantsDf["summonerName"] == "Kelody"][["matchId", "participant"]]
kelody

In [349]:
kelodyEvents = []
kelodyMatches = matchTimelineDf[(matchTimelineDf["matchId"].isin(kelody["matchId"]))]

# (matchTimelineDf["assistingParticipantIds"] == kelody["participant"].values[0]
assist = matchTimelineDf[(matchTimelineDf["matchId"].isin(kelody["matchId"])) & matchTimelineDf["assistingParticipantIds"].notna() & (matchTimelineDf["killerId"] != 8) & (matchTimelineDf["victimId"] != 8)][["assistingParticipantIds", "matchId"]]
onlyAssistDf = matchTimelineDf[(matchTimelineDf["matchId"] == "EUW1_5701667894") & (matchTimelineDf["assistingParticipantIds"].notna()) & (matchTimelineDf["killerId"] != 8) & (matchTimelineDf["victimId"] != 8)]

lyAssist = []for index, match in kelody.iterrows():
      game = []
    game = matchTimelineDf.loc[((matchTimelineDf["matchId"] == match["matchId"]) & ((matchTimelineDf["participantId"] == (match["participant"])) |
    (matchTimelineDf["creatorId"] == match["participant"]) |
    (matchTimelineDf["killerId"] == match["participant"]) |
    (matchTimelineDf["victimId"] == match["participant"])))]
    kelodyEvents.append(game)
kelodyEventsDf = pd.concat(kelodyEvents, ignore_index=True)
kelodyEventsDf

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 10)

In [358]:
kelodyEvents = []

# (matchTimelineDf["assistingParticipantIds"] == kelody["participant"].values[0]
for index, match in kelody.iterrows():
    game = []
    assist = matchTimelineDf[(matchTimelineDf["matchId"] == match["matchId"]) & matchTimelineDf["assistingParticipantIds"].notna() & (matchTimelineDf["creatorId"] != match["participant"]) & (matchTimelineDf["killerId"] != match["participant"]) & (matchTimelineDf["victimId"] != match["participant"])]
    participantList = as_list(match["participant"])
    game = matchTimelineDf.loc[((matchTimelineDf["matchId"] == match["matchId"]) & ((matchTimelineDf["participantId"] == (match["participant"])) | 
    (matchTimelineDf["creatorId"] == match["participant"]) |
    (matchTimelineDf["killerId"] == match["participant"]) |
    (matchTimelineDf["victimId"] == match["participant"]) |
    (assist["assistingParticipantIds"].apply(lambda x: any(i in x for i in participantList)))))]
    kelodyEvents.append(game)
kelodyEventsDf = pd.concat(kelodyEvents, ignore_index=True)
kelodyEventsDf

,matchId,type,participantId,itemId,timestamp,wardType,creatorId,level,killerId,victimId,bounty,positionX,positionY,assistingParticipantIds,teamId,monsterType,buildingType,laneType
0,EUW1_5701667894,ITEM_PURCHASED,8.0,1056.0,3188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EUW1_5701667894,ITEM_PURCHASED,8.0,2003.0,3188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,EUW1_5701667894,ITEM_PURCHASED,8.0,2003.0,3188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,EUW1_5701667894,LEVEL_UP,8.0,NaN,124470,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EUW1_5701667894,LEVEL_UP,8.0,NaN,156548,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1656,EUW1_5685985751,ITEM_PURCHASED,3.0,1028.0,967838,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1657,EUW1_5685985751,CHAMPION_KILL,NaN,NaN,971504,NaN,NaN,NaN,4.0,9.0,180.0,4486.0,4645.0,"[1, 2, 3, 5]",NaN,NaN,NaN,NaN
1658,EUW1_5685985751,ITEM_PURCHASED,3.0,3211.0,981084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1659,EUW1_5685985751,LEVEL_UP,3.0,NaN,1007516,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
game1Events = matchTimelineDf[(matchTimelineDf["participantId"] | matchTimelineDf["victimId"] | matchTimelineDf["killerId"] == 3)]

In [ ]:
print(timeline['info']['frames'][1])
participantsDf.to_json('participants.json', orient='records')
matchesDf.to_json('matches.json', orient='records')
matchTimelineDf.to_json('matchTimeline.json', orient='records')
